# Ag-Analytics® - Farmland Sales API 

Ag-Analytics® Farmland Sales API provides users with easy and fast access to filter and find land for sale, complete with the sales and geographic data components to get a detailed description of that land’s value. The data of this service is originally provided by [Farmland Finder](https://www.farmlandfinder.com). In this version of Land Value Service API, each parcel/land has a full description of the entire sale/transaction. Multiple parcel/land record can belong to one transaction. In geographic attribute, each result record gives the detailed description of a certain land/parcel itself, while the attribute in ‘sales’ may carry the information for the transaction/sale other than this parcel/land.

### Required libraries

In [2]:
import requests
import json
import pandas as pd
import folium
import mplleaflet
import geojson
import shapely.wkt
import os
%autosave 0

Autosave disabled


### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/farmland-sales/

       
1). __Location Parameters__(Required): 
    To make a valid request,at least one location parameters need to be provided.
    
    i. State:  The name of State in string format.Title cased.
    
    ii. County: The name of State in string format. Title cased.This parameter will be valid only if the State parameter is provided
    
    iii.Bounding_box: Area of interest in geoJSON format(See example below).
    

2). __Sale Condition Parameters__(optional): Optional parameters to specify response based on sale conditions

    i. Status: Sale Condition of the property. Vaild options are: Sold, For Sale, Expired Listing.
    
    ii. StartDate (Required only if Status is Sold):Searching starting date of the property Sale Date. In format 'yyyy-mm-dd'
    
    iii.EndDate (Required only if Status is Sold):Searching end date of the property Sale Date. In format 'yyyy-mm-dd'

### Response Description

__Listing Information__

1).Listing_id: The unique ID for each listing(transaction). 

2).Entry_Updated:The date of the sales information has been updated

3).Avg_CSR2: the average The Iowa Corn Suitability Rating(Soil Productivity Index)

4).CRP: If the property joined the Conservation Reserve Program.(‘Yes’ or ‘No’)

5).Total_Acres: the total acres of the entire sale

6).Tillable_Acres: the tillable acres of the sale

7).Percent_Tillable: the percent tillable area

8).CRP_Acres: the CRP acres

9).Sale_Price: the total sale price of the sale record

10).Price_Acre: the price per acre of the sale record

11).Status: one of the following values: "For Sale", "Listing Expired", "Sold"

12).Sale_Condition: one of the following values: "Auction", "Listing"

13).Listing_Agent: the listing agent name

14).Buyer: the buyer name as a string

15).Sale_Date: The sale date string in YYYY-MM-DD format. (When the parcel is still listing, the attribute will be Null

16).Taxes_total: taxes for the sale as a float

17).Assessed_Land: If the land/parcel has been assessed

18).Broker_URL: The URL link to the broker listing webpage as a string


__Parcel Information__

1).Parcel_ID :The unique sale id for each parcel as a string

2).Parcel: Index of the parcel in one listing(transaction).

3).Shape: The boundary of the parcel/property in Well Known Text type

4).GeoJSON: The boundary of the parcel/property in GeoJSON type

5).Acres: Area of the parcel

6).State: The state where the parcel/property locates in. 

7).County:The county where the parcel/property locates in.

8).lat_center: the latitudinal center of the parcel/property as a float value

9).lng_center: the longitudinal center of the parcel/property as a float value

10).range: the range as a string (ex: 26W - always include E or W)

11).sect: the section as string (ex: 17)

12).twnshp: the township as a string (ex: 78N - always include N or S)

13).county_name: the county name as a string (Title cased - for ex: Osceola, Polk, etc.)

14).state_name: the state name as a string (Title cased - for ex: Michigan)

15).STATEFP: the FIPS to state level as a string

16).FIPS: the FIPS to county level as a string


### Request Parameter Example

In [66]:
State= "Missouri"
County= "Audrain"
#Bounding_box = '{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-88.95973205566405,46.68995749641134],[-88.76678466796875,46.68995749641134],[-88.76678466796875,46.7981792512332],[-88.95973205566405,46.7981792512332],[-88.95973205566405,46.68995749641134]]]}}'

Status = 'For Sale'
StartDate = '2019-08-08'
EndDate = '2020-01-01'

#Format the inputs()
values = {
    'State':State,
    'County' : County,
#    'Bounding_Box': Bounding_box,
    'StartDate':StartDate,
    'EndDate':EndDate,
    'Status' :Status
}

# Header for using a subscription key.
headers = {'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

### API Function

In [67]:
def LandValueService(values,headers):
    try:
        url = 'https://agflaskprod.eastus2.cloudapp.azure.com/flask_api/api/v1.0/LandValueServiceV2/LandValue_service'
        
        response = requests.get(url, json = values,headers = headers).json()
        print(response.get('status'))
        print(response.get('msg'))
        print(response)
        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [68]:
LandValueResponse = LandValueService(values,headers)

SUCCESS
19 parcel records
{'msg': '19 parcel records', 'records': {'14A6DCEC-DC0A-49A1-93B8-B1B84032E991': {'Assessed_Land': None, 'Avg_CSR2': 50.74584, 'Broker_URL': 'https://northeastmissourirealty.com/mo/audrain/mexico/farms-ranches/65265/131320', 'Buyer': None, 'CRP': 'No', 'CRP_Acres': None, 'Entry_Updated': '2019-08-23T00:00:00', 'Listing_Agent': 'Northeast Missouri Realty', 'Listing_id': '14A6DCEC-DC0A-49A1-93B8-B1B84032E991', 'Percent_Tillable': 28.0, 'Price_Acre': 3500.0, 'Sale_Condition': 'Listing', 'Sale_Date': None, 'Sale_Price': 315000.0, 'Status': 'For Sale', 'Taxes_total': None, 'Tillable_Acres': 25.0, 'Total_Acres': 90.0, 'parcels': [{'Acres': 90.01858972522322, 'County': 'Audrain', 'FIPS': '29007', 'GeoJSON': {'coordinates': [[[-91.85805201530457, 39.15647097750574], [-91.86524033546446, 39.156362825201555], [-91.86518669128418, 39.157818707660574], [-91.86573386192322, 39.157818707660574], [-91.86568021774292, 39.16113800702124], [-91.86483263969421, 39.16085516556057

### Format Response

In [69]:
listing_info =[]
records = LandValueResponse.get('records')
for listing in records:
    parcels_info = records.get(listing).get('parcels')
    list_item = records.get(listing)
    list_item.pop('parcels')
    for parcel in parcels_info:
        parcel.update(list_item)
        listing_info.append(parcel)

listing_db = pd.DataFrame(listing_info)

### Display Response

In [70]:
#Narrow table
listing_db.set_index('Listing_id').T
#Wide table
# listing_db.set_index('Listing_id')

Listing_id,14A6DCEC-DC0A-49A1-93B8-B1B84032E991,36184CAC-FAB2-4A63-BCE5-33F618A9F5ED,46DD1C08-08F5-4E2F-AC3F-5D36C6D682AF,65B4ECFE-EDE2-48CA-9D24-D74409605F68,662528ED-5A2C-4EF9-807B-976D012E82D4,7BF0A2E0-8ED7-45DA-BC42-FD97783F7BFC,7C8DDCE4-F4B3-42A9-9725-41F13C197820,823952AB-424C-4FAA-ACF2-FE15419D9FAE,924EBAB9-5F88-4EEF-B204-ED79315AA810,9FEB171B-F9B6-4777-A723-6FFB98DDA2CE,C0811A6A-F211-4585-8535-24A77E53A180,C0811A6A-F211-4585-8535-24A77E53A180,C0811A6A-F211-4585-8535-24A77E53A180,C0811A6A-F211-4585-8535-24A77E53A180,C0811A6A-F211-4585-8535-24A77E53A180,C0811A6A-F211-4585-8535-24A77E53A180,C7896E23-08C0-4906-BA7A-80D55BD2301A,C9530F8E-EF00-440D-B772-5189ADD97434,DE9F0AFB-D0EF-42AF-9BBB-251A45140EA7
Acres,90.0186,162,82.0319,364.507,119.323,40.6804,31.1425,82.0319,66.3606,64.3031,23.1351,9.8223,5.00246,0.861657,17.2373,5.88434,160.264,39.072,46.9995
County,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain
FIPS,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007,29007
GeoJSON,"{'coordinates': [[[-91.85805201530457, 39.1564...","{'coordinates': [[[-91.7866492201864, 39.26381...","{'coordinates': [[[-91.4798211770267, 39.17784...","{'coordinates': [[[-91.85720443725586, 39.1502...","{'coordinates': [[[-91.85713469982147, 39.1501...","{'coordinates': [[[-91.856745404149, 39.312099...","{'coordinates': [[[-91.85981154441833, 39.1631...","{'coordinates': [[[-91.4798211770267, 39.17784...","{'coordinates': [[[-91.60296857357025, 39.2041...","{'coordinates': [[[-92.02504634857178, 39.1904...","{'coordinates': [[[-91.89082791105844, 39.2513...","{'coordinates': [[[-91.88772338888333, 39.2568...","{'coordinates': [[[-91.88768325296036, 39.2581...","{'coordinates': [[[-91.88770080356457, 39.2570...","{'coordinates': [[[-91.88780689554716, 39.2548...","{'coordinates': [[[-91.89069716041537, 39.2493...","{'coordinates': [[[-91.87830805778503, 39.1592...","{'coordinates': [[[-92.19027042388915, 39.3289...","{'coordinates': [[[-91.94186568260193, 39.1613..."
Parcel,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,1,1,1
Parcel_ID,5ea6fb8d526a877669abad2d,5ed127c0526a8707317c323d,5ea6fb8d526a877669abae35,5ea6fb8c526a877669aba7ff,5ea6fb8c526a877669aba76f,5ea6fb8c526a877669aba6de,5ea6fb8e526a877669abb18e,5cc99a54526a87103c364b98,5ea6fb8c526a877669aba334,5cc99a53526a87103c3647c5,5ea6fb8f526a877669abbbac,5ea6fb8f526a877669abbbad,5ea6fb8f526a877669abbbae,5ea6fb8f526a877669abbbaf,5ea6fb8f526a877669abbbb0,5ea6fb8f526a877669abbbb1,5ea6fb8c526a877669aba61e,5cc99a54526a87103c364cda,5ea6fb8c526a877669aba59f
STATEFP,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29
Shape,"POLYGON((-91.85805201530457 39.15647097750574,...","POLYGON((-91.7866492201864 39.2638133103601, -...","POLYGON((-91.4798211770267 39.1778493155894, -...","POLYGON((-91.85720443725586 39.15023114963322,...",POLYGON((-91.85713469982147 39.150122987738335...,"POLYGON((-91.856745404149 39.3120995351902, -9...","POLYGON((-91.85981154441833 39.16311786540495,...","POLYGON((-91.4798211770267 39.1778493155894, -...",POLYGON((-91.60296857357025 39.204108370864496...,POLYGON((-92.02504634857178 39.190455840387074...,"POLYGON((-91.89082791105844 39.25135529487836,...","POLYGON((-91.88772338888333 39.2568484197778, ...",POLYGON((-91.88768325296036 39.258130328153044...,POLYGON((-91.88770080356457 39.257023469555136...,"POLYGON((-91.88780689554716 39.25489880593812,...","POLYGON((-91.89069716041537 39.24930124806062,...","POLYGON((-91.87830805778503 39.15928287906368,...","POLYGON((-92.19027042388915 39.3289945923499, ...",POLYGON((-91.94186568260193 39.161329340305684...
State,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri,Missouri
county_name,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,Audrain,None,None,Non

### Visualize Parcels on Map

In [71]:
#Display parcels on map
zoom_point=listing_db['GeoJSON'][0]['coordinates'][0][0]
m = folium.Map([zoom_point[1],zoom_point[0]],tiles='Cartodb Positron', zoom_start=11,width='70%', height='100%')
for shape in listing_db['GeoJSON']:
    folium.GeoJson(shape).add_to(m)
m

### Download Response

In [83]:
download_path = '<---path on your local directory--->' # ex:r'C:\Users\YourName\Downloads'
listing_db.to_csv(os.path.join(download_path,'FarmLand_Listing.csv'))